# Création de l'agent principal

In [4]:
import duckdb
from pathlib import Path

DATA_DIR = Path("../data")
DB_PATH = DATA_DIR / "food.duckdb"

## Version initiale

In [6]:
from typing import Dict, List, Any
from dataclasses import dataclass

@dataclass
class AgentState:
    """État de l'agent pour suivre le contexte de la conversation"""
    conversation_history: List[Dict[str, Any]]
    current_context: Dict[str, Any]
    
class MainAgent:
    def __init__(self, db_path: str):
        """Initialise l'agent avec la base de données DuckDB"""
        self.conn = duckdb.connect(db_path)
        self.state = AgentState(
            conversation_history=[],
            current_context={}
        )
    
    def analyze_query(self, query: str) -> Dict[str, Any]:
        """Analyse la requête pour déterminer l'intention et les paramètres"""
        # Version simple - à améliorer avec LLM
        analysis = {
            "intent": "general_query",
            "parameters": {},
            "requires_sql": False
        }
        
        # Mots clés basiques pour détecter l'intention
        if any(word in query.lower() for word in ["combien", "nombre", "total"]):
            analysis["intent"] = "count_query"
            analysis["requires_sql"] = True
            
        return analysis
    
    def execute_query(self, analysis: Dict[str, Any]) -> str:
        """Exécute la requête en fonction de l'analyse"""
        if analysis["requires_sql"]:
            # Example simple de requête SQL
            result = self.conn.execute("SELECT COUNT(*) FROM products").fetchone()
            return f"Il y a {result[0]} produits dans la base de données."
            
        return "Je ne sais pas encore comment répondre à cette question."
    
    def process_query(self, query: str) -> str:
        """Point d'entrée principal pour traiter une requête"""
        # Analyse la requête
        analysis = self.analyze_query(query)
        
        # Mise à jour du contexte
        self.state.current_context = {
            "last_query": query,
            "analysis": analysis
        }
        
        # Exécute la requête
        response = self.execute_query(analysis)
        
        # Met à jour l'historique
        self.state.conversation_history.append({
            "query": query,
            "analysis": analysis,
            "response": response
        })
        
        return response

    def __del__(self):
        """Ferme la connexion à la base de données"""
        self.conn.close()

# Exemple d'utilisation
agent = MainAgent(DB_PATH)
response = agent.process_query("Combien y a-t-il de produits dans la base de données ?")
print(response)

Il y a 3601655 produits dans la base de données.


## Ajoutons la capacité de générer des requêtes SQL basées sur l'intention détectée

In [10]:
import duckdb
from typing import Dict, List, Any
from dataclasses import dataclass

@dataclass
class AgentState:
    """État de l'agent pour suivre le contexte de la conversation"""
    conversation_history: List[Dict[str, Any]]
    current_context: Dict[str, Any]
    
class MainAgent:
    def __init__(self, db_path: str):
        """Initialise l'agent avec la base de données DuckDB"""
        self.conn = duckdb.connect(db_path)
        self.state = AgentState(
            conversation_history=[],
            current_context={}
        )
    
    def analyze_query(self, query: str) -> Dict[str, Any]:
        """Analyse la requête pour déterminer l'intention et les paramètres"""
        query_lower = query.lower()
        analysis = {
            "intent": "general_query",
            "parameters": {},
            "requires_sql": False,
            "sql_query": None
        }
        
        if any(word in query_lower for word in ["combien", "nombre", "total"]):
            analysis["intent"] = "count_query"
            analysis["requires_sql"] = True
            analysis["sql_query"] = "SELECT COUNT(*) FROM products"
            
        elif "nutriscore" in query_lower:
            analysis["intent"] = "nutriscore_query"
            analysis["requires_sql"] = True
            analysis["sql_query"] = """
                SELECT nutriscore_grade, COUNT(*) as count 
                FROM products 
                GROUP BY nutriscore_grade 
                ORDER BY nutriscore_grade"""
                
        elif "catégories" in query_lower or "categories" in query_lower:
            analysis["intent"] = "category_query"
            analysis["requires_sql"] = True
            analysis["sql_query"] = """
                SELECT categories, COUNT(*) as count 
                FROM products 
                GROUP BY categories 
                ORDER BY count DESC 
                LIMIT 10"""
            
        return analysis
    
    def execute_query(self, analysis: Dict[str, Any]) -> str:
        """Exécute la requête en fonction de l'analyse"""
        if not analysis["requires_sql"]:
            return "Je ne sais pas encore comment répondre à cette question."
            
        result = self.conn.execute(analysis["sql_query"]).fetchall()
        
        if analysis["intent"] == "count_query":
            return f"Il y a {result[0][0]} produits dans la base de données."
            
        elif analysis["intent"] == "nutriscore_query":
            response = "Distribution des Nutriscores:\n"
            for grade, count in result:
                response += f"- Nutriscore {grade}: {count} produits\n"
            return response
            
        elif analysis["intent"] == "category_query":
            response = "Top 10 des catégories:\n"
            for category, count in result:
                response += f"- {category}: {count} produits\n"
            return response
            
        return "Je ne sais pas comment formater cette réponse."
    
    def process_query(self, query: str) -> str:
        """Point d'entrée principal pour traiter une requête"""
        # Analyse la requête
        analysis = self.analyze_query(query)
        
        # Mise à jour du contexte
        self.state.current_context = {
            "last_query": query,
            "analysis": analysis
        }
        
        # Exécute la requête
        response = self.execute_query(analysis)
        
        # Met à jour l'historique
        self.state.conversation_history.append({
            "query": query,
            "analysis": analysis,
            "response": response
        })
        
        return response

    def __del__(self):
        """Ferme la connexion à la base de données"""
        self.conn.close()

# Exemple d'utilisation
agent = MainAgent(DB_PATH)
print(agent.process_query("Combien y a-t-il de produits dans la base de données ?"))
print(agent.process_query("Combien y a-t-il de produits ?"))
print(agent.process_query("Montre-moi la distribution des nutriscores"))
print(agent.process_query("Quelles sont les principales catégories ?"))


Il y a 3601655 produits dans la base de données.
Il y a 3601655 produits dans la base de données.
Distribution des Nutriscores:
- Nutriscore a: 161848 produits
- Nutriscore b: 133265 produits
- Nutriscore c: 245111 produits
- Nutriscore d: 305603 produits
- Nutriscore e: 326479 produits
- Nutriscore not-applicable: 73417 produits
- Nutriscore unknown: 2334933 produits
- Nutriscore None: 20999 produits

Top 10 des catégories:
- None: 1925020 produits
- : 139033 produits
- Snacks: 33766 produits
- Snacks, Sweet snacks, Confectioneries: 14597 produits
- Condiments, Sauces, Groceries: 13532 produits
- Dairies, Fermented foods, Fermented milk products, Cheeses: 12018 produits
- Snacks, Sweet snacks, Biscuits and cakes, Biscuits: 11143 produits
- Beverages: 9528 produits
- Desserts, Frozen foods, Frozen desserts: 9520 produits
- Plant-based foods and beverages, Plant-based foods, Cereals and potatoes, Breads: 7888 produits



## Examiner la structure de la base DuckDB

In [12]:
import duckdb
from pathlib import Path

DB_PATH = Path("../data") / "food.duckdb"
con = duckdb.connect(str(DB_PATH))

# Afficher le schéma
schema_query = """
SELECT 
    column_name,
    data_type,
    is_nullable
FROM information_schema.columns 
WHERE table_name = 'products'
ORDER BY column_name;
"""

# Afficher quelques statistiques de base
stats_query = """
SELECT
    COUNT(*) as total_rows,
    COUNT(DISTINCT nutriscore_grade) as unique_nutriscores,
    COUNT(DISTINCT categories) as unique_categories
FROM products;
"""

print("=== Schéma de la table ===")
print(con.execute(schema_query).df())

print("\n=== Statistiques de base ===")
print(con.execute(stats_query).df())

con.close()

=== Schéma de la table ===
                       column_name  data_type is_nullable
0                      additives_n    INTEGER         YES
1                   additives_tags  VARCHAR[]         YES
2                   allergens_tags  VARCHAR[]         YES
3                           brands    VARCHAR         YES
4                      brands_tags  VARCHAR[]         YES
..                             ...        ...         ...
104          unknown_ingredients_n    INTEGER         YES
105         unknown_nutrients_tags  VARCHAR[]         YES
106                  vitamins_tags  VARCHAR[]         YES
107  with_non_nutritive_sweeteners    INTEGER         YES
108                with_sweeteners    INTEGER         YES

[109 rows x 3 columns]

=== Statistiques de base ===
   total_rows  unique_nutriscores  unique_categories
0     3601655                   7             245672


## Ollama avec Mistral-7B

Le code ci-dessous montre comment poser une question simple à Mistral-7B et obtenir une réponse.

Les choix de paramètres de génération sont décrits [ici](https://github.com/ollama/ollama/blob/main/docs/modelfile.md#valid-parameters-and-values).

In [20]:
from ollama import Client
from IPython.display import Markdown, display

# Initialiser Ollama
ollama = Client(host='http://localhost:11434')

def simple_question(question: str) -> str:
    """Fonction pour poser une question simple à Ollama"""
    try:
        response = ollama.chat(
            model='mistral:7b',
            messages=[{'role': 'user', 'content': question}],
            options={
                'temperature': 0.4  # Contrôle la créativité (0-1)
            }
        )
        return response['message']['content'].strip()
        
    except Exception as e:
        print(f"Error classifying question: {e}")

prompt = "Bonjour Ollama, connais-tu DuckDB?"
response = simple_question(prompt)
display(Markdown(response))


Bonjour! Oui, je suis familière avec DuckDB. C'est un système de gestion de bases de données en temps réel et open source écrit en Rust. Il est optimisé pour le traitement de petites à moyennes quantités de données et offre une performance comparable à celle de SQLite, mais avec des fonctionnalités supplémentaires telles que l'intégration de moteurs de machine learning et la possibilité d'exécuter du code Rust natif.

In [ ]:
from ollama import Client
from typing import Dict

class LLMAgent:
    def __init__(self):
        """Initialise la connexion à Ollama"""
        self.client = Client(host='http://localhost:11434')
        self.model = 'mistral:7b'
        
    def generate_sql(self, query: str, schema: Dict) -> str:
        """Génère une requête SQL à partir d'une question en langage naturel"""
        prompt = f"""En tant qu'expert SQL, génère une requête SQL sur une base DuckDB pour répondre à cette question: "{query}"

Schéma de la table products:
{schema}

Retourne uniquement la requête SQL, sans explications."""
        
        response = self.client.chat(model=self.model, messages=[{
            'role': 'user',
            'content': prompt
        }])
        
        return response['message']['content'].strip()

    def classify_query(self, query: str) -> Dict:
        """Classifie le type de requête"""
        prompt = f"""Classifie cette question: "{query}"
        
Retourne uniquement un objet JSON avec:
- intent: le type de requête (count, category, nutriscore, etc.)
- requires_sql: true/false si une requête SQL est nécessaire

Example: {{"intent": "count_query", "requires_sql": true}}"""
        
        response = self.client.chat(model=self.model, messages=[{
            'role': 'user',
            'content': prompt
        }])
        
        # Note: Il faudrait ajouter la validation du JSON retourné
        return response['message']['content']

Je comprends que le modèle LLM doit connaître et comprendre le schéma de la base DuckDB pour généer la requête SQL à partir d'une question en langage naturel.

Mais, je ne la comprend pas moi-même. Il faudrait peut-être que j'examine la base DuckDB afin d'éliminer les colonnes inutiles et documenter les colonnes restantes. Comment faire cela?

Voici une requête pour analyser le schéma et obtenir des statistiques basiques sur chaque colonne :

In [23]:
import duckdb
from pathlib import Path

DB_PATH = Path("../data/food.duckdb")
con = duckdb.connect(str(DB_PATH))

# Obtenir le schéma
schema_query = """
SELECT column_name, data_type, is_nullable
FROM information_schema.columns 
WHERE table_name = 'products'
ORDER BY column_name;
"""

# Statistiques de base
stats_query = """
SELECT
    COUNT(*) as total_rows,
    COUNT(DISTINCT product_name) as unique_products,
    COUNT(DISTINCT nutriscore_grade) as unique_nutriscores
FROM products;
"""

print("=== Schéma ===")
print(con.execute(schema_query).df())
print("\n=== Stats de base ===")
print(con.execute(stats_query).df())

=== Schéma ===
                       column_name  data_type is_nullable
0                      additives_n    INTEGER         YES
1                   additives_tags  VARCHAR[]         YES
2                   allergens_tags  VARCHAR[]         YES
3                           brands    VARCHAR         YES
4                      brands_tags  VARCHAR[]         YES
..                             ...        ...         ...
104          unknown_ingredients_n    INTEGER         YES
105         unknown_nutrients_tags  VARCHAR[]         YES
106                  vitamins_tags  VARCHAR[]         YES
107  with_non_nutritive_sweeteners    INTEGER         YES
108                with_sweeteners    INTEGER         YES

[109 rows x 3 columns]

=== Stats de base ===
   total_rows  unique_products  unique_nutriscores
0     3601655          2305601                   7


In [ ]:
from pathlib import Path
import duckdb
import pandas as pd

def analyze_schema():
    DB_PATH = Path("../data/food.duckdb")
    con = duckdb.connect(str(DB_PATH))
   
    # 1. Obtenir le schéma
    schema_query = """
    SELECT column_name, data_type, is_nullable
    FROM information_schema.columns 
    WHERE table_name = 'products'
    ORDER BY column_name;
    """
    schema = con.execute(schema_query).df()
   
    # 2. Pour chaque colonne
    results = []
    for _, row in schema.iterrows():
        col_name = row['column_name']
        stats = {
            'name': col_name,
            'type': row['data_type'],
            'nullable': row['is_nullable'],
            'stats': {}
        }
       
        # Statistiques de base
        basic_stats = con.execute(f"""
            SELECT 
                COUNT(*) as total,
                COUNT(DISTINCT {col_name}) as unique_values,
                COUNT(*) - COUNT({col_name}) as null_count
            FROM products
        """).fetchone()
       
        stats['stats'] = {
            'total': basic_stats[0],
            'unique_values': basic_stats[1],
            'null_count': basic_stats[2]
        }
       
        # Échantillon de valeurs
        if col_name.endswith('_tags'):
            # Pour les colonnes de type tableau
            sample = con.execute(f"""
                WITH expanded AS (
                    SELECT DISTINCT elem
                    FROM products, 
                            UNNEST({col_name}) AS elem
                    WHERE {col_name} IS NOT NULL
                    LIMIT 20
                )
                SELECT elem
                FROM expanded
                ORDER BY elem
            """).fetchall()
            stats['sample_values'] = [str(x[0]) for x in sample]
        else:
            # Pour les colonnes normales
            sample = con.execute(f"""
                SELECT DISTINCT {col_name}
                FROM products
                WHERE {col_name} IS NOT NULL
                ORDER BY {col_name}
                LIMIT 20
            """).fetchall()
            stats['sample_values'] = [str(x[0]) for x in sample]
            
        results.append(stats)
   
    # Générer le markdown
    md = "# Dictionnaire des données\n\n"
    for col in results:
        md += f"## {col['name']}\n"
        md += f"- Type: {col['type']}\n"
        md += f"- Nullable: {col['nullable']}\n"
        md += f"- Total rows: {col['stats']['total']}\n"
        md += f"- Unique values: {col['stats']['unique_values']}\n"
        md += f"- Null count: {col['stats']['null_count']}\n"
        if 'sample_values' in col:
            md += f"- Sample values: {', '.join(col['sample_values'])}\n"
        md += "\n"
   
    # Sauvegarder
    Path("../docs/markdown/data_dictionary.md").write_text(md)
   
    con.close()

analyze_schema()